In [149]:
from datasrcs.spc import load_tor_segments
from datasrcs.stormevents import load_tor_events

import pandas as pd
import numpy as np

%env WORKDIR=~/weatherpy-work

env: WORKDIR=~/weatherpy-work


In [11]:
spc_segs = load_tor_segments(to_tz='CST')
spc_03 = spc_segs[spc_segs.yr == 2003]

stevents_03 = load_tor_events('2003-01-01', '2003-12-31', tz='CST')

In [14]:
spc_03.columns

Index(['date_time', 'yr', 'mo', 'dy', 'tz', 'st', 'stf', 'stn', 'mag', 'inj',
       'fat', 'loss', 'closs', 'slat', 'slon', 'elat', 'elon', 'len', 'wid',
       'ns', 'sn', 'sg', 'f1', 'f2', 'f3', 'f4', 'fc'],
      dtype='object')

In [154]:
def tojoinable(data, multiplier=100):
    return (data * 100).astype(int)

In [162]:
spc_cols = ['date_time', 'slat1', 'slon1', 'elat1', 'elon1', 'len', 'mag', 'f1', 'f2']
spc_03[['slat1', 'slon1', 'elat1', 'elon1']] = tojoinable(spc_subset[['slat', 'slon', 'elat', 'elon']])
spc_subset = spc_03[spc_cols]
spc_subset.sort_values(by='date_time').head()

/Users/jimtang/miniconda3/envs/storm_analysis_env2/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,date_time,slat1,slon1,elat1,elon1,len,mag,f1,f2
om,,,,,,,,,
9,2003-02-15 12:00:00,2995,-9200,2995,-9200,0.5,1,113,0
688,2003-02-15 16:20:00,3213,-8978,3215,-8978,1.0,0,121,0
689,2003-02-15 16:43:00,3218,-8960,3227,-8953,3.5,0,129,123
690,2003-02-15 17:25:00,3235,-8927,3236,-8923,2.0,1,101,0
4,2003-02-15 20:36:00,3233,-8782,3236,-8777,2.5,1,91,0


In [166]:
stevent_cols = ['begin_date_time', 'end_date_time', 'begin_lat1', 'begin_lon1', 'end_lat1', 'end_lon1', 'event_id']
stevents_03[['begin_lat', 'begin_lon', 'end_lat', 'end_lon']] = stevents_03[['begin_lat', 'begin_lon', 'end_lat', 'end_lon']].round(2)
stevents_03[['begin_lat1', 'begin_lon1', 'end_lat1', 'end_lon1']] = tojoinable(stevents_03[['begin_lat', 'begin_lon', 'end_lat', 'end_lon']])
stevents_subset = stevents_03[stevent_cols]
stevents_subset.sort_values(by='begin_date_time').head()

,begin_date_time,end_date_time,begin_lat1,begin_lon1,end_lat1,end_lon1,event_id
226,2003-02-15 12:00:00,2003-02-15 12:02:00,2995,-9200,2995,-9200,5344180
161,2003-02-15 16:20:00,2003-02-15 16:22:00,3213,-8978,3215,-8978,5342364
192,2003-02-15 16:43:00,2003-02-15 16:48:00,3218,-8960,3228,-8958,5342466
193,2003-02-15 16:48:00,2003-02-15 16:53:00,3222,-8955,3225,-8953,5342467
187,2003-02-15 17:25:00,2003-02-15 17:27:00,3235,-8927,3236,-8923,5342469


In [167]:
merged = spc_subset.merge(stevents_subset, 
             left_on=['date_time', 'slat1', 'slon1'], 
             right_on=['begin_date_time', 'begin_lat1', 'begin_lon1'],
             how='outer', indicator=True)

In [191]:
merged_sorted = merged.sort_values('date_time')
criteria = merged_sorted.date_time.between('2003-02-15 12:00', '2003-02-16 12:00')
merged_sorted.loc[criteria, spc_cols + stevent_cols + ['_merge']]

,date_time,slat1,slon1,elat1,elon1,len,mag,f1,f2,begin_date_time,end_date_time,begin_lat1,begin_lon1,end_lat1,end_lon1,event_id,_merge
0,2003-02-15 12:00:00,2995.0,-9200.0,2995.0,-9200.0,0.5,1.0,113.0,0.0,2003-02-15 12:00:00,2003-02-15 12:02:00,2995.0,-9200.0,2995.0,-9200.0,5344180.0,both
1,2003-02-15 16:20:00,3213.0,-8978.0,3215.0,-8978.0,1.0,0.0,121.0,0.0,2003-02-15 16:20:00,2003-02-15 16:22:00,3213.0,-8978.0,3215.0,-8978.0,5342364.0,both
2,2003-02-15 16:43:00,3218.0,-8960.0,3227.0,-8953.0,3.5,0.0,129.0,123.0,2003-02-15 16:43:00,2003-02-15 16:48:00,3218.0,-8960.0,3228.0,-8958.0,5342466.0,both
3,2003-02-15 17:25:00,3235.0,-8927.0,3236.0,-8923.0,2.0,1.0,101.0,0.0,2003-02-15 17:25:00,2003-02-15 17:27:00,3235.0,-8927.0,3236.0,-8923.0,5342469.0,both
4,2003-02-15 20:36:00,3233.0,-8782.0,3236.0,-8777.0,2.5,1.0,91.0,0.0,2003-02-15 20:36:00,2003-02-15 20:40:00,3233.0,-8782.0,3236.0,-8777.0,5340389.0,both
5,2003-02-16 04:00:00,3007.0,-8558.0,3007.0,-8558.0,0.2,1.0,5.0,0.0,2003-02-16 04:00:00,2003-02-16 04:00:00,3007.0,-8558.0,3007.0,-8558.0,5342682.0,both
6,2003-02-16 04:30:00,3063.0,-8572.0,3063.0,-8572.0,0.2,0.0,133.0,0.0,2003-02-16 04:30:00,2003-02-16 04:30:00,3063.0,-8572.0,3063.0,-8572.0,5342779.0,both
7,2003-02-16 05:37:00,2982.0,-8530.0,2982.0,-8530.0,0.2,0.0,45.0,0.0,2003-02-16 05:37:00,2003-02-16 05:37:00,2982.0,-8530.0,2982.0,-8530.0,5342778.0,both
8,2003-02-16 07:25:00,3103.0,-8420.0,3112.0,-8422.0,2.4,1.0,131.0,205.0,2003-02-16 07:25:00,2003-02-16 07:26:00,3103.0,-8420.0,3103.0,-8420.0,5342780.0,both


In [168]:
core = merged[merged._merge == 'both'] 

In [169]:
spconly = merged.loc[merged._merge == 'left_only', spc_cols]
steventsonly = merged.loc[merged._merge == 'right_only', stevent_cols]

In [170]:
# del steventsonly['_merge']
spcwithf2 = spconly.loc[spconly.f2 != 0, spc_cols]
end_merged = steventsonly.merge(spcwithf2, left_on=['end_lat1', 'end_lon1'], 
                                right_on=['elat1', 'elon1'], indicator=True, how='outer')

In [172]:
segments = end_merged[end_merged._merge == 'both']
outliers_stevents = end_merged[end_merged._merge == 'left_only']
outliers_spc = end_merged[end_merged._merge == 'right_only']

In [173]:
core

,date_time,slat1,slon1,elat1,elon1,len,mag,f1,f2,begin_date_time,end_date_time,begin_lat1,begin_lon1,end_lat1,end_lon1,event_id,_merge
0,2003-02-15 12:00:00,2995.0,-9200.0,2995.0,-9200.0,0.5,1.0,113.0,0.0,2003-02-15 12:00:00,2003-02-15 12:02:00,2995.0,-9200.0,2995.0,-9200.0,5344180.0,both
1,2003-02-15 16:20:00,3213.0,-8978.0,3215.0,-8978.0,1.0,0.0,121.0,0.0,2003-02-15 16:20:00,2003-02-15 16:22:00,3213.0,-8978.0,3215.0,-8978.0,5342364.0,both
2,2003-02-15 16:43:00,3218.0,-8960.0,3227.0,-8953.0,3.5,0.0,129.0,123.0,2003-02-15 16:43:00,2003-02-15 16:48:00,3218.0,-8960.0,3228.0,-8958.0,5342466.0,both
3,2003-02-15 17:25:00,3235.0,-8927.0,3236.0,-8923.0,2.0,1.0,101.0,0.0,2003-02-15 17:25:00,2003-02-15 17:27:00,3235.0,-8927.0,3236.0,-8923.0,5342469.0,both
4,2003-02-15 20:36:00,3233.0,-8782.0,3236.0,-8777.0,2.5,1.0,91.0,0.0,2003-02-15 20:36:00,2003-02-15 20:40:00,3233.0,-8782.0,3236.0,-8777.0,5340389.0,both
5,2003-02-16 04:00:00,3007.0,-8558.0,3007.0,-8558.0,0.2,1.0,5.0,0.0,2003-02-16 04:00:00,2003-02-16 04:00:00,3007.0,-8558.0,3007.0,-8558.0,5342682.0,both
6,2003-02-16 04:30:00,3063.0,-8572.0,3063.0,-8572.0,0.2,0.0,133.0,0.0,2003-02-16 04:30:00,2003-02-16 04:30:00,3063.0,-8572.0,3063.0,-8572.0,5342779.0,both
7,2003-02-16 05:37:00,2982.0,-8530.0,2982.0,-8530.0,0.2,0.0,45.0,0.0,2003-02-16 05:37:00,2003-02-16 05:37:00,2982.0,-8530.0,2982.0,-8530.0,5342778.0,both
8,2003-02-16 07:25:00,3103.0,-8420.0,3112.0,-8422.0,2.4,1.0,131.0,205.0,2003-02-16 07:25:00,2003-02-16 07:26:00,3103.0,-8420.0,3103.0,-8420.0,5342780.0,both
9,2003-02-21 23:35:00,3120.0,-8723.0,3120.0,-8723.0,0.1,0.0,53.0,0.0,2003-02-21 23:35:00,2003-02-21 23:37:00,3120.0,-8723.0,3120.0,-8723.0,5343209.0,both


In [174]:
segments

,begin_date_time,end_date_time,begin_lat1,begin_lon1,end_lat1,end_lon1,event_id,date_time,slat1,slon1,elat1,elon1,len,mag,f1,f2,_merge
8,2003-07-13 13:30:00,2003-07-13 13:45:00,3475.0,-8263.0,3475.0,-8253.0,5329643.0,2003-07-13 13:20:00,3479.0,-8270.0,3475.0,-8253.0,12.0,1.0,77.0,7.0,both
13,2003-05-09 14:55:00,2003-05-09 15:00:00,3675.0,-7733.0,3672.0,-7731.0,5331061.0,2003-05-09 14:50:00,3675.0,-7733.0,3672.0,-7731.0,4.0,0.0,175.0,175.0,both
49,2003-05-14 20:40:00,2003-05-14 20:50:00,3322.0,-9422.0,3315.0,-9407.0,5353104.0,2003-05-14 19:45:00,3368.0,-9470.0,3315.0,-9407.0,51.8,2.0,37.0,67.0,both
54,2003-05-06 22:35:00,2003-05-06 23:02:00,3660.0,-8880.0,3653.0,-8850.0,5352476.0,2003-05-06 22:32:00,3661.0,-8885.0,3653.0,-8850.0,22.0,1.0,105.0,83.0,both
56,2003-05-07 00:01:00,2003-05-07 00:18:00,3272.0,-9463.0,3282.0,-9452.0,5354154.0,2003-05-06 22:45:00,3268.0,-9545.0,3282.0,-9452.0,54.4,2.0,499.0,459.0,both
63,2003-04-15 18:30:00,2003-04-15 18:36:00,3452.0,-9987.0,3453.0,-9978.0,5352590.0,2003-04-15 18:29:00,3450.0,-9983.0,3453.0,-9978.0,5.5,1.0,65.0,57.0,both
84,2003-04-19 23:47:00,2003-04-20 00:11:00,3625.0,-9247.0,3642.0,-9228.0,5350982.0,2003-04-19 23:43:00,3620.0,-9252.0,3642.0,-9228.0,16.0,1.0,89.0,5.0,both
113,2003-05-04 18:50:00,2003-05-04 19:03:00,3707.0,-9342.0,3713.0,-9337.0,5358175.0,2003-05-04 17:50:00,3692.0,-9420.0,3713.0,-9337.0,49.0,3.0,145.0,109.0,both
141,2003-05-16 18:42:00,2003-05-16 18:44:00,3486.0,-9207.0,3488.0,-9205.0,5360794.0,2003-05-16 18:35:00,3479.0,-9217.0,3488.0,-9205.0,8.2,0.0,119.0,85.0,both
144,2003-05-07 13:47:00,2003-05-07 13:49:00,3318.0,-8647.0,3318.0,-8645.0,5361201.0,2003-05-07 13:45:00,3320.0,-8647.0,3318.0,-8645.0,1.3,1.0,117.0,121.0,both


In [176]:
outliers_spc[spc_cols]

,date_time,slat1,slon1,elat1,elon1,len,mag,f1,f2
260,2003-05-06 20:32:00,3727.0,-8903.0,3738.0,-8848.0,33.0,4.0,153.0,127.0


In [182]:
steventsonly[stevent_cols].sort_values('begin_date_time')

,begin_date_time,end_date_time,begin_lat1,begin_lon1,end_lat1,end_lon1,event_id
1449,2003-02-15 16:48:00,2003-02-15 16:53:00,3222.0,-8955.0,3225.0,-8953.0,5342467.0
1447,2003-02-16 07:26:00,2003-02-16 07:30:00,3107.0,-8422.0,3112.0,-8422.0,5342781.0
1509,2003-03-06 02:51:00,2003-03-06 02:57:00,3378.0,-8562.0,3375.0,-8558.0,5348836.0
1452,2003-03-20 04:25:00,2003-03-20 04:30:00,3137.0,-8403.0,3140.0,-8395.0,5347028.0
1448,2003-03-27 13:20:00,2003-03-27 13:25:00,2618.0,-81408.0,2617.0,-8175.0,5344665.0
1446,2003-03-27 16:20:00,2003-03-27 16:22:00,2598.0,-8015.0,2598.0,-8015.0,5344777.0
1451,2003-03-29 17:25:00,2003-03-29 17:25:00,3852.0,-77002.0,3852.0,-77002.0,5347621.0
1501,2003-04-04 15:11:00,2003-04-04 15:11:00,4025.0,-8923.0,4025.0,-8923.0,5350606.0
1455,2003-04-04 15:48:00,2003-04-04 15:48:00,4018.0,-8897.0,4018.0,-8897.0,5350405.0
1456,2003-04-04 15:53:00,2003-04-04 15:53:00,4018.0,-8882.0,4018.0,-8882.0,5350406.0


In [179]:
criteria = stevents_03.begin_date_time.between('2003-02-15 16:00', '2003-02-16 12:00')
stevents_03[criteria].sort_values('begin_date_time')[stevent_cols]

,begin_date_time,end_date_time,begin_lat1,begin_lon1,end_lat1,end_lon1,event_id
161,2003-02-15 16:20:00,2003-02-15 16:22:00,3213,-8978,3215,-8978,5342364
192,2003-02-15 16:43:00,2003-02-15 16:48:00,3218,-8960,3228,-8958,5342466
193,2003-02-15 16:48:00,2003-02-15 16:53:00,3222,-8955,3225,-8953,5342467
187,2003-02-15 17:25:00,2003-02-15 17:27:00,3235,-8927,3236,-8923,5342469
150,2003-02-15 20:36:00,2003-02-15 20:40:00,3233,-8782,3236,-8777,5340389
191,2003-02-16 04:00:00,2003-02-16 04:00:00,3007,-8558,3007,-8558,5342682
176,2003-02-16 04:30:00,2003-02-16 04:30:00,3063,-8572,3063,-8572,5342779
170,2003-02-16 05:37:00,2003-02-16 05:37:00,2982,-8530,2982,-8530,5342778
167,2003-02-16 07:25:00,2003-02-16 07:26:00,3103,-8420,3103,-8420,5342780
168,2003-02-16 07:26:00,2003-02-16 07:30:00,3107,-8422,3112,-8422,5342781
